In [21]:
from importlib import reload
from mcs import mcs_module
import cobra
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")

In [22]:
from mcs import mcs_module
reload(mcs_module)

<module 'mcs.mcs_module' from 'c:\\users\\phili\\documents\\python\\mcs\\mcs\\mcs_module.py'>

In [23]:
mcs_mod = mcs_module.MCS_Module(ecc2,"desired","lin_constraints","-1 Growth <= -0.5\n 1 AcEx >= 2")

NameError: Equations must contain a sign (<=,=,>=)

In [2]:
import sys
import scipy.sparse as sp
import numpy as np
import cplex as cp

csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].

In [7]:
from cplex.exceptions import CplexError

my_rows = [0, 0, 0, 0, 1, 1, 1, 2, 2]
my_cols = [0, 1, 2, 3, 0, 1, 2, 1, 3]
my_vals = [-1.0, 1.0, 1.0, 10.0, 1.0, -3.0, 1.0, 1.0, -3.5]

# data common to all populateby functions
my_obj = [1.0, 2.0, 3.0, 1.0]
my_ub = [40.0, np.Infinity, np.Infinity, 3.0]
my_lb = [0.0, 0.0, 0.0, 2.0]
my_ctype = "CCCI"
my_colnames = ["x1", "x2", "x3", "x4"]
my_rhs = [20.0, 30.0, 0.0]
my_A = sp.csc_matrix((my_vals,(my_rows,my_cols)))
my_rownames = ["r1", "r2", "r3"]
my_sense = "LLE"

# csr, csc, lil formats

def populatebynonzero(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    prob.linear_constraints.add(rhs=my_rhs, senses=my_sense,
                                names=my_rownames)
    prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype,
                       names=my_colnames)

    # retrieve row and column indices from sparse matrix and convert them to int
    indices_A = [[int(j) for j in i] for i in my_A.nonzero()]
    prob.linear_constraints.set_coefficients(zip(indices_A[0], indices_A[1], list(my_A.data)))

try:
    my_prob = cp.Cplex()
    handle = populatebynonzero(my_prob)
except CplexError as exc:
    print(exc)
print()
my_prob.solve()
# solution.get_status() returns an integer code
print("Solution status = ", my_prob.solution.get_status(), ":", end=' ')
# the following line prints the corresponding string
print(my_prob.solution.status[my_prob.solution.get_status()])
print("Solution value  = ", my_prob.solution.get_objective_value())

numcols = my_prob.variables.get_num()
numrows = my_prob.linear_constraints.get_num()

slack = my_prob.solution.get_linear_slacks()
x = my_prob.solution.get_values()

for j in range(numrows):
    print("Row %d:  Slack = %10f" % (j, slack[j]))
for j in range(numcols):
    print("Column %d:  Value = %10f" % (j, x[j]))


Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 1 rows and 2 columns.
MIP Presolve modified 3 coefficients.
Aggregator did 2 substitutions.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.01 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.01 ticks)
Solution status =  101 : MIP_optimal
Solution value  =  90.94999999999999
Row 0:  Slack =   0.000000
Row 1:  Slack =   0.000000
Row 2:  Slack =   0.000000
Column 0:  Value =   0.500000
Column 1:  Value =   1.050000
Column 2:  Value =  28.450000
Column 3:  Value =   3.000000
